In [1]:
#How to use the trained model
#you have to create the same CNN which you have trained
#Inorder to use the trained model, you have to create replicate the model which you have trained. 
#Image should be preprocessed in the way which it is fed during the training. 
#The following note book captures how well the model is behaving on different classification of the fruits. 

import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import tensorflow as tf
import keras
import glob
import cv2
import os
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.layers import LSTM, Input, TimeDistributed
from keras.models import Model
from keras.optimizers import RMSprop, SGD
# Import the backend
from keras import backend as K

/Users/sriramreddy/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
fruit_images = []
labels = [] 
for fruit_dir_path in glob.glob("/Users/sriramreddy/Downloads/523/ex2/train/*"):
    fruit_label = fruit_dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (110, 110))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) 
        #why do we need to convert the RGB2BGR
        #I don't think it is going to affect training
        #BGR was a choice made for historical reasons and now we have to live with it. In other words, BGR is the horse’s ass in OpenCV.
        fruit_images.append(image)
        labels.append(fruit_label)
fruit_images = np.array(fruit_images)
labels = np.array(labels)
label_to_id_dict = {v:i for i,v in enumerate(np.unique(labels))}
id_to_label_dict = {v: k for k, v in label_to_id_dict.items()}
label_ids = np.array([label_to_id_dict[x] for x in labels])

In [3]:
validation_fruit_images = []
validation_labels = [] 
for fruit_dir_path in glob.glob("/Users/sriramreddy/Downloads/523/ex2/test/*"):
    fruit_label = fruit_dir_path.split("/")[-1]
    for image_path in glob.glob(os.path.join(fruit_dir_path, "*.png")):
        image = cv2.imread(image_path, cv2.IMREAD_COLOR)
        image = cv2.resize(image, (110, 110))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        validation_fruit_images.append(image)
        validation_labels.append(fruit_label)
validation_fruit_images = np.array(validation_fruit_images)
validation_labels = np.array(validation_labels)
validation_label_ids = np.array([label_to_id_dict[x] for x in validation_labels])
validation_fruit_images.shape, validation_label_ids.shape

((2698, 110, 110, 3), (2698,))

In [4]:
X_train, X_test = fruit_images, validation_fruit_images
Y_train, Y_test = label_ids, validation_label_ids

#Normalize color values to between 0 and 1
X_train = X_train/255
X_test = X_test/255

#Make a flattened version for some of our models
X_flat_train = X_train.reshape(X_train.shape[0], 110*110*3)
X_flat_test = X_test.reshape(X_test.shape[0], 110*110*3)

#One Hot Encode the Output what is this 60 
Y_train = keras.utils.to_categorical(Y_train, 60)
Y_test = keras.utils.to_categorical(Y_test, 60)

print('Original Sizes:', X_train.shape, X_test.shape, Y_train.shape, Y_test.shape)
print('Flattened:', X_flat_train.shape, X_flat_test.shape)

Original Sizes: (10901, 110, 110, 3) (2698, 110, 110, 3) (10901, 60) (2698, 60)
Flattened: (10901, 36300) (2698, 36300)


In [5]:
model_cnn = Sequential()
# First convolutional layer, note the specification of shape
model_cnn.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(110, 110, 3)))
model_cnn.add(Conv2D(64, (3, 3), activation='relu'))
model_cnn.add(MaxPooling2D(pool_size=(2, 2)))
model_cnn.add(Dropout(0.25))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation='relu'))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(60, activation='softmax'))

model_cnn.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model_cnn.fit(X_train, Y_train,
          batch_size=128,
          epochs=1,
          verbose=1,
          validation_data=(X_test, Y_test))
score = model_cnn.evaluate(X_test, Y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 10901 samples, validate on 2698 samples
Epoch 1/1
10901/10901 [==============================] - 548s 50ms/step - loss: 1.7621 - acc: 0.4621 - val_loss: 0.8072 - val_acc: 0.6709
Test loss: 0.8072312606361903
Test accuracy: 0.6708673091178651


In [6]:
#I'm loading the weights
fname="/Users/sriramreddy/Downloads/523/ex2/weights_cnn_classification_augmented_data.hdf5"
model_cnn.load_weights(fname)
score = model_cnn.evaluate(X_test, Y_test, verbose=0)

In [7]:
print('Test loss after loading weights:', score[0])
print('Test accuracy after loading weights:', score[1])

Test loss after loading weights: 0.18243249262955427
Test accuracy after loading weights: 0.949592290585619


In [9]:
#Prediction of an image
import numpy as np
from keras.preprocessing import image
test_image = image.load_img('/Users/sriramreddy/Downloads/523/ex2/load_test/apple.png', target_size = (110, 110))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
test_image = test_image/255
result = model_cnn.predict(test_image)
print(np.shape(result),result)

(1, 60) [[4.1474420e-01 2.8747948e-20 3.7118767e-12 5.8525580e-01 1.5474299e-22
  7.1599966e-14 3.2037425e-18 8.9590591e-21 5.6318555e-22 1.2083604e-17
  5.8864034e-19 1.2959383e-22 5.4487309e-18 3.0047478e-20 3.1845395e-18
  2.2890178e-18 2.3104935e-18 5.8596469e-21 6.2133083e-23 1.2725730e-18
  3.2080334e-19 4.2164237e-22 2.9573556e-17 1.7499023e-19 1.7644623e-18
  3.1610383e-16 5.5884330e-22 1.5699200e-18 3.7210717e-19 1.4817105e-19
  2.2796020e-22 1.7397926e-18 6.4252643e-20 8.0155621e-17 1.5872732e-20
  1.4147785e-22 3.9447578e-20 8.3771277e-18 1.8371097e-19 3.1737080e-19
  2.2650330e-20 5.4304321e-17 1.6882786e-19 9.7760827e-19 5.0910492e-20
  1.5623847e-20 1.2897741e-21 4.7148546e-18 1.5278072e-18 7.4605190e-20
  8.1016509e-21 3.2781519e-19 4.3955756e-21 2.9675345e-19 2.7854661e-19
  4.2911510e-20 2.9735176e-19 9.0334294e-20 6.5976695e-18 8.6656276e-18]]


In [10]:
id_labels={0: 'freshapples',
 1: 'freshbanana',
 2: 'freshoranges',
 3: 'rottenapples',
 4: 'rottenbanana',
 5: 'rottenoranges'}
print(id_labels.items())

dict_items([(0, 'freshapples'), (1, 'freshbanana'), (2, 'freshoranges'), (3, 'rottenapples'), (4, 'rottenbanana'), (5, 'rottenoranges')])


In [12]:
accuracy={}
infi,infi_set=0,0
for k,v in id_labels.items():
    total_set,count=0,0
    for i in glob.glob("/Users/sriramreddy/Downloads/523/ex2/test/"+v+"/*png"):
        test_image = image.load_img(i, target_size = (110, 110))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        test_image = test_image/255
        result = model_cnn.predict(test_image)
        total_set+=1
        infi_set+=1
        if result[0][k] >= 0.5:
            count+=1
            infi+=1
    accuracy[k]=count/total_set
    print(v,accuracy[k])
print("Accuracy for all items", infi/infi_set)

freshapples 0.830379746835443
freshbanana 0.989501312335958
freshoranges 0.9149484536082474
rottenapples 0.9900166389351082
rottenbanana 0.9943396226415094
rottenoranges 0.9305210918114144
Accuracy for all items 0.9477390659747962


In [ ]:
Augmented test data set
    freshapples 0.5037974683544304
    freshbanana 1.0
    freshoranges 0.9072164948453608
    rottenapples 0.6339434276206323
    rottenbanana 0.36792452830188677
    rottenoranges 0.47146401985111663
Accuracy for all items 0.6293550778354337

In [40]:
accuracy={}
infi,infi_set=0,0
for k,v in id_labels.items():
    total_set,count=0,0
    for i in glob.glob("/Users/sriramreddy/Downloads/523/original_data_set/"+v+"/*png"):
        test_image = image.load_img(i, target_size = (110, 110))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model_cnn.predict(test_image)
        total_set+=1
        infi_set+=1
        if result[0][k] >= 0.5:
            count+=1
            infi+=1
    accuracy[k]=count/total_set
    print(v,accuracy[k])
print("Accuracy for all items", infi/infi_set)

freshapples 0.5517241379310345
freshbanana 1.0
freshoranges 0.9466019417475728
rottenapples 0.6391437308868502
rottenbanana 0.43790849673202614
rottenoranges 0.5135135135135135
Accuracy for all items 0.6604897418927862


In [ ]:
The results for the original data set
    freshapples 0.5431034482758621
    freshbanana 1.0
    freshoranges 0.9466019417475728
    rottenapples 0.636085626911315
    rottenbanana 0.434640522875817
    rottenoranges 0.5045045045045045
Accuracy for all items 0.6604897418927862

In [41]:
accuracy={}
infi,infi_set=0,0
for k,v in id_labels.items():
    total_set,count=0,0
    for i in glob.glob("/Users/sriramreddy/Downloads/523/augmented_data_set/"+v+"/*png"):
        test_image = image.load_img(i, target_size = (110, 110))
        test_image = image.img_to_array(test_image)
        test_image = np.expand_dims(test_image, axis = 0)
        result = model_cnn.predict(test_image)
        total_set+=1
        infi_set+=1
        if result[0][k] >= 0.5:
            count+=1
            infi+=1
    accuracy[k]=count/total_set
    print(v,accuracy[k])
print("Accuracy for all items", infi/infi_set)

freshapples 0.5478927203065134
freshbanana 1.0
freshoranges 0.9374325782092773
rottenapples 0.6860346585117227
rottenbanana 0.36782861292665214
rottenoranges 0.474974974974975
Accuracy for all items 0.6489447753511287
